# Data & Algoritma Understanding

## Data Understanding

### 📊 Nama Dataset
liputan6_data.tar.gz

### 🌍 Languages
- Indonesian

### 🧩 Data Structure
|Nama Kolom|Tipe Data|
|----|--------|
|`id`|`string`|
|`url`|`string`|
|`clean_article`|`string`|
|`clean_summary`|`string`|
|`extractive_summary`|`string`|

### Data Instances
|Nama Kolom|Contoh Data|
|----------|-----------|
|`id`|26408|
|`url`|https://www.liputan6.com/news/read/26408/pbb-siap-membantu-penyelesaian-konflik-ambon|
|`clean_article`|Liputan6.com, Ambon: Partai Bulan Bintang wilayah Maluku bertekad membantu pemerintah menyelesaikan konflik di provinsi tersebut. Syaratnya, penanganan penyelesaian konflik Maluku harus dimulai dari awal kerusuhan, yakni 19 Januari 1999. Demikian hasil Musyawarah Wilayah I PBB Maluku yang dimulai Sabtu pekan silam dan berakhir Senin (31/12) di Ambon. Menurut seorang fungsionaris PBB Ridwan Hasan, persoalan di Maluku bisa selesai asalkan pemerintah dan aparat keamanan serius menangani setiap persoalan di Maluku secara komprehensif dan bijaksana. Itulah sebabnya, PBB wilayah Maluku akan menjadikan penyelesaian konflik sebagai agenda utama partai. PBB Maluku juga akan mendukung penegakan hukum secara terpadu dan tanpa pandang bulu. Siapa saja yang melanggar hukum harus ditindak. Ridwan berharap, Ketua PBB Maluku yang baru, Ali Fauzi, dapat menindak lanjuti agenda politik partai yang telah diamanatkan dan mau mendukung penegakan hukum di Maluku. (ULF/Sahlan Heluth).|
|`clean_summary`|Konflik Ambon telah berlangsung selama tiga tahun. Partai Bulan Bintang wilayah Maluku siap membantu pemerintah menyelesaikan kasus di provinsi tersebut.|
|`extractive_summary`|Liputan6.com, Ambon: Partai Bulan Bintang wilayah Maluku bertekad membantu pemerintah menyelesaikan konflik di provinsi tersebut. Siapa saja yang melanggar hukum harus ditindak.|

### Data Fields
|Nama Kolom|Keterangan|
|----------|----------|
|`id`|Kolom id unique|
|`url`|URL Article|
|`clean_article`|Isi original article|
|`clean_summary`|Ringkasan Abstract|
|`extractive_summary`|Ringkasan Ekstractif|

## Algoritma Understanding

# Model Training & Evaluation

### Load Dataset dan Convert Data


In [1]:
import pandas as pd


In [15]:
df_train = pd.read_csv('../data/train_data.csv')
df_train.head()

,id,url,clean_article,clean_summary,extractive_summary
0,100000,https://www.liputan6.com/news/read/100000/yudh...,"[['Liputan6', '.', 'com', ',', 'Jakarta', ':',...","[['Menurut', 'Presiden', 'Susilo', 'Bambang', ...","[0, 1]"
1,100002,https://www.liputan6.com/news/read/100002/jepa...,"[['Liputan6', '.', 'com', ',', 'Jakarta', ':',...","[['Pada', 'masa', 'silam', 'Jepang', 'terlalu'...","[2, 3]"
2,100003,https://www.liputan6.com/news/read/100003/pulu...,"[['Liputan6', '.', 'com', ',', 'Kutai', ':', '...","[['Puluhan', 'hektare', 'areal', 'persawahan',...","[1, 5]"
3,100004,https://www.liputan6.com/news/read/100004/pres...,"[['Liputan6', '.', 'com', ',', 'Jakarta', ':',...","[['Sekjen', 'PBB', 'Kofi', 'Annan', 'memuji', ...","[2, 5]"
4,100005,https://www.liputan6.com/news/read/100005/warg...,"[['Liputan6', '.', 'com', ',', 'Solok', ':', '...","[['Untuk', 'mempercepat', 'pelaksanaan', 'bela...","[0, 2]"


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193883 entries, 0 to 193882
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id                  193883 non-null  int64 
 1   url                 193883 non-null  object
 2   clean_article       193883 non-null  object
 3   clean_summary       193883 non-null  object
 4   extractive_summary  193883 non-null  object
dtypes: int64(1), object(4)
memory usage: 7.4+ MB


## Process


In [2]:
import pandas as pd
import ast
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertModel

### Memuat Data dan Preprocessing

In [3]:
# Konversi string yang berisi list menjadi list
def parse_list_string(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return []

# Load dataset
print("Memuat data...")
df_train = pd.read_csv('../data/train2_data.csv')

# Cleaning data
print("Membersihkan data...")
df_train['clean_article'] = df_train['clean_article'].apply(parse_list_string)
df_train['extractive_summary'] = df_train['extractive_summary'].apply(parse_list_string)


print("Membagi data menjadi set Training (60%), Validasi (20%), dan Testing (20%)...")

# Ambil data validasi 20% dari data training
df_train_val, df_test_articles = train_test_split(
    df_train,
    test_size=0.2,
    random_state=42
)

# Ambil data testing 25% dari data training
df_train_articles, df_val_articles = train_test_split(
    df_train_val,
    test_size=0.25,
    random_state=42
)

print(f"Jumlah artikel untuk Training: {len(df_train_articles)}")
print(f"Jumlah artikel untuk Validasi: {len(df_val_articles)}")
print(f"Jumlah artikel untuk Testing: {len(df_test_articles)}")


# Fungsi untuk mengubah format dari per-artikel menjadi per-kalimat
def create_sentence_dataframe(df):
    processed_data = []
    for _, row in df.iterrows():
        article_sentences_tokens = row['clean_article']
        summary_indices = set(row['extractive_summary'])
        sentences_as_strings = [" ".join(sent) for sent in article_sentences_tokens]
        for i, sent_text in enumerate(sentences_as_strings):
            processed_data.append({
                'sentence': sent_text,
                'label': 1 if i in summary_indices else 0,
            })
    return pd.DataFrame(processed_data)

# Buat DataFrame per-kalimat untuk setiap set secara terpisah
print("\nMengubah data menjadi format per-kalimat...")
train_sentences_df = create_sentence_dataframe(df_train_articles)
val_sentences_df = create_sentence_dataframe(df_val_articles)
test_sentences_df = create_sentence_dataframe(df_test_articles)

print(f"Jumlah kalimat di data Training: {len(train_sentences_df)}")
print(f"Jumlah kalimat di data Validasi: {len(val_sentences_df)}")
print(f"Jumlah kalimat di data Testing: {len(test_sentences_df)}")


Memuat data...
Membersihkan data...
Membagi data menjadi set Training (60%), Validasi (20%), dan Testing (20%)...
Jumlah artikel untuk Training: 6582
Jumlah artikel untuk Validasi: 2195
Jumlah artikel untuk Testing: 2195

Mengubah data menjadi format per-kalimat...
Jumlah kalimat di data Training: 76399
Jumlah kalimat di data Validasi: 25819
Jumlah kalimat di data Testing: 26240


In [ ]:
# Hitung jumlah setiap nilai unik di kolom 'label'
class_distribution = train_sentences_df['label'].value_counts()

print("Distribusi Kelas pada Data Training:")
print(class_distribution)

# Jika ingin melihat dalam bentuk persentase
print("\nDistribusi Kelas dalam Persentase:")
class_percentage = train_sentences_df['label'].value_counts(normalize=True) * 100
print(class_percentage)

Distribusi Kelas pada Data Training:
label
0    62562
1    13837
Name: count, dtype: int64

Distribusi Kelas dalam Persentase:
label
0    81.888506
1    18.111494
Name: proportion, dtype: float64


### Tokenisasi dengan IndoBERT Tokenizer

In [10]:
MODEL_NAME = 'indobenchmark/indobert-base-p1'
MAX_LEN = 128

print(f"\nMemuat tokenizer dari {MODEL_NAME}...")
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

def convert_to_bert_input(sentences, tokenizer, max_len):
    return tokenizer.batch_encode_plus(
        sentences.tolist(),
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True
    )

print("\nMelakukan tokenisasi pada semua dataset...")
# Tokenisasi semua set data secara terpisah
X_train_tokens = convert_to_bert_input(train_sentences_df['sentence'], tokenizer, MAX_LEN)
X_val_tokens = convert_to_bert_input(val_sentences_df['sentence'], tokenizer, MAX_LEN)
X_test_tokens = convert_to_bert_input(test_sentences_df['sentence'], tokenizer, MAX_LEN)

# Ambil label untuk setiap set secara terpisah
y_train = train_sentences_df['label'].values
y_val = val_sentences_df['label'].values
y_test = test_sentences_df['label'].values


Memuat tokenizer dari indobenchmark/indobert-base-p1...

Melakukan tokenisasi pada semua dataset...


In [15]:
print("\nMenghitung bobot kelas untuk mengatasi data tidak seimbang...")
# Hitung bobotnya
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
# Ubah menjadi format dictionary yang bisa dibaca Keras
class_weight_dict = dict(enumerate(class_weights))

print(f"Bobot Kelas yang akan digunakan: {class_weight_dict}")


Menghitung bobot kelas untuk mengatasi data tidak seimbang...
Bobot Kelas yang akan digunakan: {0: 0.6105862983919952, 1: 2.760677892606779}


### MEMBANGUN DAN MELATIH MODEL IndoBERT

In [15]:
# pip install tf-keras

In [6]:
def build_model(bert_model, max_len=128):
    input_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    sequence_output = bert_model(input_ids, attention_mask=attention_mask)[0]
    cls_token_output = sequence_output[:, 0, :]
    x = tf.keras.layers.Dropout(0.2)(cls_token_output)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output_layer)
    return model

print("\nMemuat pre-trained IndoBERT model...")
bert_model = TFBertModel.from_pretrained(MODEL_NAME)
model = build_model(bert_model, max_len=MAX_LEN)
model.summary()

optimizer = tf.keras.optimizers.AdamW(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [
    tf.keras.metrics.BinaryAccuracy('accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    verbose=1,
    mode='min',
    restore_best_weights=True
)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='indobert_summarizer_best_model.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

BATCH_SIZE = 64
EPOCHS = 5 

print("\nMemulai training model...")

# Konversi input menjadi format dictionary yang bisa dibaca model
train_input_dict = {'input_ids': tf.constant(X_train_tokens['input_ids']), 'attention_mask': tf.constant(X_train_tokens['attention_mask'])}
val_input_dict = {'input_ids': tf.constant(X_val_tokens['input_ids']), 'attention_mask': tf.constant(X_val_tokens['attention_mask'])}

history = model.fit(
    train_input_dict,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(val_input_dict, y_val),
    callbacks=[early_stopping, model_checkpoint],
    class_weight=class_weight_dict
)

print("Training selesai.")


Memuat pre-trained IndoBERT model...



Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1244413   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   44         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                       

C:\Users\SUWAAAN\AppData\Roaming\Python\Python312\site-packages\tf_keras\src\engine\training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1194/1194 [==============================] - 14221s 12s/step - loss: 0.6563 - accuracy: 0.6352 - precision: 0.2717 - recall: 0.6036 - val_loss: 0.6312 - val_accuracy: 0.6632 - val_precision: 0.2949 - val_recall: 0.6413
Epoch 2/5
1194/1194 [==============================] - ETA: 0s - loss: 0.5948 - accuracy: 0.6818 - precision: 0.3200 - recall: 0.6728
Epoch 2: val_loss improved from 0.63119 to 0.59770, saving model to indobert_summarizer_best_model.h5
1194/1194 [==============================] - 12938s 11s/step - loss: 0.5948 - accuracy: 0.6818 - precision: 0.3200 - recall: 0.6728 - val_loss: 0.5977 - val_accuracy: 0.6835 - val_precision: 0.3065 - val_recall: 0.6156
Epoch 3/5
1194/1194 [==============================] - ETA: 0s - loss: 0.5208 - accuracy: 0.7301 - precision: 0.3785 - recall: 0.7635
Epoch 3: val_loss did not improve from 0.59770
1194/1194 [==============================] - 13000s 11s/step - loss: 0.5208 - accuracy: 0.7301 - precision: 0.3785 - recall: 0.7635 - val_loss: 0

### Evaluasi Model pada Data Test

In [16]:
print("\nMemuat model terbaik dari checkpoint untuk evaluasi...")

best_model = tf.keras.models.load_model(
    'indobert_summarizer_best_model-melihat_class_weight.h5', 
    custom_objects={"TFBertModel": TFBertModel}
)

threshold = 0.4

# Siapkan data test
test_input_dict = {'input_ids': tf.constant(X_test_tokens['input_ids']), 'attention_mask': tf.constant(X_test_tokens['attention_mask'])}

print("\nMelakukan evaluasi pada Test Set (data yang belum pernah dilihat)...")
y_pred_proba = best_model.predict(test_input_dict)
y_pred = (y_pred_proba > threshold).astype(int)

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred, target_names=['Bukan Ringkasan (0)', 'Ringkasan (1)']))


Memuat model terbaik dari checkpoint untuk evaluasi...


Melakukan evaluasi pada Test Set (data yang belum pernah dilihat)...
820/820 [==============================] - 1311s 2s/step

Classification Report (Test Set):
                     precision    recall  f1-score   support

Bukan Ringkasan (0)       0.92      0.48      0.63     21610
      Ringkasan (1)       0.25      0.80      0.38      4630

           accuracy                           0.54     26240
          macro avg       0.58      0.64      0.50     26240
       weighted avg       0.80      0.54      0.59     26240



In [17]:
sample_article_data = df_test_articles['clean_article'].iloc[0]
sample_sentences = [" ".join(sent) for sent in sample_article_data]

In [18]:
print(sample_article_data)

[['Liputan6', '.', 'com', ',', 'Jakarta', ':', 'Pengamat', 'politik', 'Andi', 'Malarangeng', ',', 'baru-baru', 'ini', ',', 'menilai', 'pembaharuan', 'kode', 'etik', 'DPR', 'tidak', 'akan', 'berarti', 'banyak', 'dalam', 'mengubah', 'etika', 'berpolitik', 'para', 'anggota', 'Dewan', '.'], ['Sebab', ',', 'pengaturan', 'masalah', 'etika', 'belum', 'ditetapkan', 'dalam', 'Undang-undang', '.'], ['Menurut', 'Andi', ',', 'muatan', 'kode', 'etik', 'DPR', 'yang', 'akan', 'disampaikan', 'pekan', 'ini', 'masih', 'terlihat', 'samar', '.'], ['Selain', 'itu', ',', 'hubungan', 'antara', 'Pasal', 'Satu', 'dan', 'lainnya', 'tak', 'memiliki', 'konsekuensi', 'yang', 'jelas', '.'], ['Akibatnya', ',', 'kode', 'etik', 'tersebut', 'hanya', 'akan', 'menimbulkan', 'perdebatan', '.'], ['Bahkan', ',', 'tak', 'layak', 'menjadi', 'acuan', 'pokok', 'kedisiplinan', 'dan', 'etika', 'politik', 'para', 'anggota', 'DPR', ',', 'terutama', 'soal', 'batasan', 'hibah', ',', 'sangsi', 'kedisiplinan', 'hingga', 'pembentukan', 

In [23]:
print(sample_sentences)

['Liputan6 . com , Jakarta : Pengamat politik Andi Malarangeng , baru-baru ini , menilai pembaharuan kode etik DPR tidak akan berarti banyak dalam mengubah etika berpolitik para anggota Dewan .', 'Sebab , pengaturan masalah etika belum ditetapkan dalam Undang-undang .', 'Menurut Andi , muatan kode etik DPR yang akan disampaikan pekan ini masih terlihat samar .', 'Selain itu , hubungan antara Pasal Satu dan lainnya tak memiliki konsekuensi yang jelas .', 'Akibatnya , kode etik tersebut hanya akan menimbulkan perdebatan .', 'Bahkan , tak layak menjadi acuan pokok kedisiplinan dan etika politik para anggota DPR , terutama soal batasan hibah , sangsi kedisiplinan hingga pembentukan Dewan Kehormatan Angota DPR .', 'Karena itu , Andi menilai sudah seharusnya kode etik tersebut dapat dilepaskan dari pengaturan UU .', 'Sebab , etika DPR dan pejabat negara , hingga kini , masih belum diatur dalam UU .', 'Menanggapi hal tersebut , Ketua Komisi II DPR Teras Narang menyatakan , dalam penegakan kod

### INFERENCE (MEMBUAT RINGKASAN)

In [24]:
sample_article_data = df_test_articles['clean_article'].iloc[1]
sample_sentences = [" ".join(sent) for sent in sample_article_data]

In [25]:
def summarize_article_with_threshold(article_sentences, model, tokenizer, max_len, threshold=threshold):
    inputs = tokenizer.batch_encode_plus(
        article_sentences,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf',
        add_special_tokens=True
    )
    input_dict = {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']}
    
    # Dapatkan probabilitas untuk setiap kalimat
    probabilities = model.predict(input_dict).flatten()
    
    # Saring kalimat berdasarkan threshold
    summary_indices = []
    for i, p in enumerate(probabilities):
        if p > threshold:
            summary_indices.append(i)
    
    summary = " ".join([article_sentences[i] for i in summary_indices])
    
    # Jika tidak ada kalimat yang lolos, kembalikan kalimat dengan skor tertinggi
    if not summary:
        print("Tidak ada kalimat yang melewati threshold, mengambil 1 kalimat terbaik.")
        best_index = np.argmax(probabilities)
        summary = article_sentences[best_index]
        
    return summary


print("--- Artikel Asli (5 kalimat pertama) ---")
for sent in sample_sentences[:5]:
    print("- " + sent)

print(f"\n--- Ringkasan yang Dihasilkan Model (dengan Threshold = {threshold}) ---")
generated_summary_threshold = summarize_article_with_threshold(
    sample_sentences, 
    best_model, 
    tokenizer, 
    max_len=MAX_LEN, 
    threshold=threshold 
)
print(generated_summary_threshold)

--- Artikel Asli (5 kalimat pertama) ---
- Liputan6 . com , Jakarta : Pemerintah harus menyelesaikan Kasus Aceh secara konsisten dan komprehensif .
- Pertumbuhan separatis Gerakan Aceh Merdeka dan rasa ketidakadilan antarwarga dinilai penyebab bermacam masalah di Serambi Mekah itu .
- Hal itu mengemuka dalam Rapat Kerja antara Komisi I DPR dan Panglima TNI Laksamana Widodo A . S .
- di Jakarta , Senin ( 3/12 ) .
- Widodo menilai , upaya dialog yang dilakukan selama ini tak sempurna .

--- Ringkasan yang Dihasilkan Model (dengan Threshold = 0.4) ---
1/1 [==============================] - 1s 545ms/step
Liputan6 . com , Jakarta : Pemerintah harus menyelesaikan Kasus Aceh secara konsisten dan komprehensif . Pertumbuhan separatis Gerakan Aceh Merdeka dan rasa ketidakadilan antarwarga dinilai penyebab bermacam masalah di Serambi Mekah itu . Widodo menilai , upaya dialog yang dilakukan selama ini tak sempurna . Sebab , komponen yang hadir tidak mencerminkan keseluruhan aspirasi rakyat Aceh . 